In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import numpy as np

In [2]:
pip install tqdm

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [3]:
import numpy as np

In [4]:
dtypes = {'Sub-Phylotype': str, 'Sequevar': str, 'Alt Names': str}
df = pd.read_csv('C:/Users/PC/Desktop/ralstonia final database copy.csv')                  

In [5]:
df.head()

,Index,Phylotype,Sub-Phylotype,Sequevar,Sub-sequevar designation,Strainname,Host Species (Common name),Host Species,Host Genus,Host Family,Host Order,Location Isolated,Location (Country or Territory)
0,1,I,NaN,12,NaN,M10,Morus alba (Mulberry),Morus alba,Morus,Moraceae,Rosales,"Zhejiang, China",China
1,2,I,NaN,12,NaN,M11,Morus alba (Mulberry),Morus alba,Morus,Moraceae,Rosales,"Zhejiang, China",China
2,3,I,NaN,12,NaN,R288,Morus alba (Mulberry),Morus alba,Morus,Moraceae,Rosales,China,China
3,4,I,NaN,12,NaN,R292,Morus alba (Mulberry),Morus alba,Morus,Moraceae,Rosales,China,China
4,5,I,NaN,12,NaN,RUN90,Morus alba (Mulberry),Morus alba,Morus,Moraceae,Rosales,"Guangdong, China",China


In [6]:
selected_phylotypes = ['I', 'II', 'III', 'IV']
filtered_df = df[df['Phylotype'].isin(selected_phylotypes)].copy()

In [7]:
host_common_name_extract = filtered_df['Host Species (Common name)'].str.extract(r'\(([^)]+)\)|^((?![\s\S]*\().*[\s\S]*(?![\s\S]*\))$)')
filtered_df['host_common_name'] = host_common_name_extract[0].fillna(host_common_name_extract[1])

In [8]:
def preprocess_host_common_name(name):
    if pd.isnull(name):
        return name
    name = name.lower().strip()
    name = ' '.join(name.split())
    return name
filtered_df['host_common_name'] = host_common_name_extract[0].fillna(host_common_name_extract[1])
filtered_df['host_common_name'] = filtered_df['host_common_name'].apply(preprocess_host_common_name)


In [9]:
phylotype_pivot = pd.pivot_table(filtered_df, index='host_common_name', columns='Phylotype', values='Strainname', aggfunc='count', fill_value=0)
phylotype_pivot_log10 = phylotype_pivot.applymap(lambda x: np.log10(x) if x > 0 else 0)


In [10]:
sequevar_pivot = pd.pivot_table(filtered_df, index='host_common_name', columns='Sequevar', values='Strainname', aggfunc='count', fill_value=0)
sequevar_pivot_log10 = sequevar_pivot.applymap(lambda x: np.log10(x) if x > 0 else 0)

In [11]:
plt.figure(figsize=(100, 100))
ax = sns.heatmap(phylotype_pivot_log10, fmt='.2f', cmap='YlGnBu', linewidths=0.5, cbar=False)
for i, row in enumerate(phylotype_pivot.index):
    for j, col in enumerate(phylotype_pivot.columns):
        ax.text(j + 0.5, i + 0.5, int(phylotype_pivot.at[row, col]),
                horizontalalignment='center',
                verticalalignment='center',
                fontsize=12)
plt.title('Phylotype Distribution across Host Species')
plt.xlabel('Phylotype')
plt.ylabel('Host Common Name')
plt.show()

In [12]:
plt.figure(figsize=(100, 100))
ax = sns.heatmap(sequevar_pivot_log10, fmt='.2f', cmap='YlGnBu', linewidths=0.5, cbar=False)
for i, row in enumerate(sequevar_pivot.index):
    for j, col in enumerate(sequevar_pivot.columns):
        ax.text(j + 0.5, i + 0.5, int(sequevar_pivot.at[row, col]),
                horizontalalignment='center',
                verticalalignment='center',
                fontsize=12)
plt.title('Sequevar Distribution across Host Species')
plt.xlabel('Sequevar')
plt.ylabel('Host Common Name')
plt.show()

In [13]:
import pandas as pd
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
from functools import lru_cache
import concurrent.futures
import plotly.graph_objs as go
from plotly.subplots import make_subplots
from tqdm import tqdm

geolocator = Nominatim(user_agent="my_geocoder", timeout=20)
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1, max_retries=10, error_wait_seconds=10)

#cache lat/lon results for each location
@lru_cache(maxsize=None)
def get_lat_lon(location):
    try:
        loc = geocode(location)
        return loc.latitude, loc.longitude
    except:
        return None, None

def process_location(location):
    lat, lon = get_lat_lon(location)
    return lat, lon

# get unique locations
unique_locations = df['Location Isolated'].unique()

# geocoding progress bar
with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
    lat_lon_list = list(tqdm(executor.map(process_location, unique_locations), total=len(unique_locations)))

# create dictionary of unique locations and their latitudes and longitudes
location_dict = dict(zip(unique_locations, lat_lon_list))

# map latitude and longitude values to the original df
df['latitude'] = df['Location Isolated'].map(lambda x: location_dict[x][0])
df['longitude'] = df['Location Isolated'].map(lambda x: location_dict[x][1])

# filter for NA values in sequevar & phlotypes in original df
grouped_df = df.groupby(['Location Isolated', 'latitude', 'longitude', 'Phylotype']).agg({
    'Strainname': lambda x: ', '.join(x.astype(str)),
    'Sequevar': lambda x: ', '.join(x.astype(str))
}).reset_index()

100%|██████████████████████████████████████████████████████████████████████████████| 1229/1229 [20:46<00:00,  1.01s/it]


In [63]:
for location, (lat, lon) in location_dict.items():
    if lat is None or lon is None:
        print(f"Missing coordinates for location: {location}")

Missing coordinates for location: Yangzi River Basin
Missing coordinates for location: Appromponou, Ivory Coast
Missing coordinates for location: Utsonomiya, Tochigi, Japan
Missing coordinates for location: Martinique Island, French West Indes, France
Missing coordinates for location: Fort. do Tabocão, Tocantins, Brazil
Missing coordinates for location: Sainte Luce, Martinique Island, French West Indes, France
Missing coordinates for location: Saint François, Guadeloupe Island, French West Indes, 
Missing coordinates for location: Guadeloupe Island, French West Indes, 
Missing coordinates for location: Huaraz, Aneash, Peru
Missing coordinates for location: Imported to Apopka, Florida, United States
Missing coordinates for location: Imported to Davie, Florida, United States
Missing coordinates for location: Imported to Florida, United States
Missing coordinates for location: Imported to Winter Garden, Florida, United States
Missing coordinates for location: Imported to Miami, Florida, U

In [60]:
def test_location(location):
    lat, lon = get_lat_lon(location)
    print(f"Location: {location} -> Latitude: {lat}, Longitude: {lon}")

test_location("China")

Location: China -> Latitude: 35.000074, Longitude: 104.999927


In [43]:
# Replace incorrect location name with the updated location name
df.loc[df['Location Isolated'] == 'Sanmenjiang, Guangxi, China', 'Location Isolated'] = 'Sanmen Jiang, Guangxi, China'
df.loc[df['Location Isolated'] == 'Obala (ZAE5), Cameroon', 'Location Isolated'] = 'Obala, Cameroon'
df.loc[df['Location Isolated'] == 'Hsinchu, Northwest, Taiwan', 'Location Isolated'] = 'Hsinchu, Taiwan'
df.loc[df['Location Isolated'] == 'Yaounde (ZAE5), Cameroon', 'Location Isolated'] = 'Yaounde, Cameroon'
df.loc[df['Location Isolated'] == 'Bafia (ZAE5), Cameroon', 'Location Isolated'] = 'Bafia, Cameroon'
df.loc[df['Location Isolated'] == 'Djebonoua-Kondoukro, Ivory Coast', 'Location Isolated'] = "N'Djébonouan, Ivory Coast"
df.loc[df['Location Isolated'] == 'Datta, Houphouékro, Ivory Coast', 'Location Isolated'] = 'Divo, Ivory Coast'
df.loc[df['Location Isolated'] == 'Yunlin, Central, Taiwan', 'Location Isolated'] = 'Yunlin, Taiwan'
df.loc[df['Location Isolated'] == 'Taitung, South East, Taiwan', 'Location Isolated'] = 'Taitung, Taiwan'
df.loc[df['Location Isolated'] == 'Huai River Basin, China', 'Location Isolated'] = 'Huai River Basin'
df.loc[df['Location Isolated'] == 'Honshu, Shikoku, Kyushu, Japan', 'Location Isolated'] = 'Japan'
df.loc[df['Location Isolated'] == 'Tres Ceibas, Jutiapa, Guatemala', 'Location Isolated'] = 'Jutiapa, Guatemala'
df.loc[df['Location Isolated'] == 'Los Llanos, Sanarate, Guatemala', 'Location Isolated'] = 'Sanarate, Guatemala'
df.loc[df['Location Isolated'] == 'Yangzi River Basin, China', 'Location Isolated'] = 'Yangzi River Basin'
df.loc[df['Location Isolated'] == 'Ilan, East, Taiwan', 'Location Isolated'] = 'Ilan, Taiwan'
df.loc[df['Location Isolated'] == 'Tainan, Southwest, Taiwan', 'Location Isolated'] = 'Tainan, Taiwan'
df.loc[df['Location Isolated'] == 'Hokkaido, Honshu, Shikoku, Kyushu, Okinawa, Japan', 'Location Isolated'] = 'Japan'
df.loc[df['Location Isolated'] == 'Mayotte Island, France', 'Location Isolated'] = 'Mayotte, France'
df.loc[df['Location Isolated'] == 'Shikoku, Kyoshu', 'Location Isolated'] = 'Shikoku, Japan'
df.loc[df['Location Isolated'] == 'Xuanen, Hubei, China', 'Location Isolated'] = "Xuan'en, China"
df.loc[df['Location Isolated'] == 'Fuijan, China', 'Location Isolated'] = 'Fujian, China'
df.loc[df['Location Isolated'] == 'Souandala P1, Ivory Coast', 'Location Isolated'] = 'Sandala, Ivory Coast'
df.loc[df['Location Isolated'] == 'Pakobo, Tiassalé, Ivory Coast', 'Location Isolated'] = 'Pakobo, Ivory Coast'
df.loc[df['Location Isolated'] == 'Attobro, Appromponou, Ivory Coast', 'Location Isolated'] = 'Appromponou, Ivory Coast'
df.loc[df['Location Isolated'] == 'Sinfra-Bas-fond, Ivory Coast', 'Location Isolated'] = 'Sinfra, Ivory Coast'
df.loc[df['Location Isolated'] == 'Abomey-Calavi area (Atlantic District), Benin', 'Location Isolated'] = 'Abomey Calavi, Benin'
df.loc[df['Location Isolated'] == 'Biankouma-Souandala , Ivory Coast', 'Location Isolated'] = 'Biankouma, Ivory Coast'
df.loc[df['Location Isolated'] == 'Analamanga (or Vohidrotra, Tamatave?), Madagascar', 'Location Isolated'] = 'Analamanga, Madagascar'
df.loc[df['Location Isolated'] == 'Minamiarima, Minamishimabara, Nagasaki, Japan', 'Location Isolated'] = 'Minamishimabara, Nagasaki Japan'
df.loc[df['Location Isolated'] == 'Ogawara, Hirado, Nagasaki, Japan', 'Location Isolated'] = 'Hirado, Nagasaki, Japan'
df.loc[df['Location Isolated'] == 'Chinzei, Karatsu, Saga, Japan', 'Location Isolated'] = 'Karatsu, Saga, Japan'
df.loc[df['Location Isolated'] == 'Shimosakiyama, Goto, Nagasaki, Japan', 'Location Isolated'] = 'Goto, Nagasaki, Japan'
df.loc[df['Location Isolated'] == 'Nonokire, Goto, Nagasaki, Japan', 'Location Isolated'] = 'Goto, Nagasaki, Japan'
df.loc[df['Location Isolated'] == 'Tsutsumi, Goto, Nagasaki, Japan', 'Location Isolated'] = 'Goto, Nagasaki, Japan'
df.loc[df['Location Isolated'] == 'Seihi, Saikai, Nagasaski, Japan', 'Location Isolated'] = 'Nagasaki Prefecture, Japan'
df.loc[df['Location Isolated'] == 'Toyotama, Tsushima, Nagasaki, Japan', 'Location Isolated'] = 'Tsushima, Nagasaki, Japan'
df.loc[df['Location Isolated'] == 'Kamiagata, Tsushima, Nagasaki, Japan', 'Location Isolated'] = 'Tsushima, Nagasaki, Japan'
df.loc[df['Location Isolated'] == 'Bezzerros, PE, Brazil', 'Location Isolated'] = 'Bezerros, Brazil'
df.loc[df['Location Isolated'] == 'Le Lamentin, Martinique Island, French West Indes, France', 'Location Isolated'] = 'Le Lamentin, France'
df.loc[df['Location Isolated'] == 'Schoelcher, Martinique Island, French West Indes, France', 'Location Isolated'] = 'Schoelcher, France'
df.loc[df['Location Isolated'] == 'Vieux Habitants, Guadeloupe Island, French West Indes,', 'Location Isolated'] = 'Vieux Habitants, Guadeloupe'
df.loc[df['Location Isolated'] == 'Fort de France, Martinique Island, French West Indes, France', 'Location Isolated'] = 'Fort de France, France'
df.loc[df['Location Isolated'] == 'Macouba, Martinique Island, French West Indes, France', 'Location Isolated'] = 'Macouba, France'
df.loc[df['Location Isolated'] == 'Saint Joseph, Martinique Island, French West Indes, France', 'Location Isolated'] = 'Saint-Joseph, France'
df.loc[df['Location Isolated'] == 'Bellefontaine, Martinique Island, French West Indes, France', 'Location Isolated'] = 'Bellefontaine, France'
df.loc[df['Location Isolated'] == 'Ducos, Martinique Island, French West Indes, France', 'Location Isolated'] = 'Ducos, France'
df.loc[df['Location Isolated'] == 'Marigot, Martinique Island, French West Indes, France', 'Location Isolated'] = 'Marigot, France'
df.loc[df['Location Isolated'] == 'Morne des Esses, Martinique Island, French West Indes, France', 'Location Isolated'] = 'Morne des Esses. France'
df.loc[df['Location Isolated'] == 'Gros Morne, Martinique Island, French West Indes, France', 'Location Isolated'] = 'Gros Morne, France'
df.loc[df['Location Isolated'] == 'Lamentin, Martinique Island, French West Indes, France', 'Location Isolated'] = 'Lamentin, France'
df.loc[df['Location Isolated'] == 'St Joseph, Martinique Island, French West Indes, France', 'Location Isolated'] = 'Saint-Joseph, France'
df.loc[df['Location Isolated'] == 'Le Lorrain, Martinique Island, French West Indes, France', 'Location Isolated'] = 'Le Lorrain, France'
df.loc[df['Location Isolated'] == 'Le Marin, Martinique Island, French West Indes, France', 'Location Isolated'] = 'Le Marin, France'
df.loc[df['Location Isolated'] == 'Morne Vert, Martinique Island, French West Indes, France', 'Location Isolated'] = 'Morne Vert, France'
df.loc[df['Location Isolated'] == 'Sainte Anne, Martinique Island, French West Indes, France', 'Location Isolated'] = 'Sainte Anne, France'
df.loc[df['Location Isolated'] == 'Trinité, Martinique Island, French West Indes, France', 'Location Isolated'] = 'Trinité, France'
df.loc[df['Location Isolated'] == 'Morne Rouge, Martinique Island, French West Indes, France', 'Location Isolated'] = 'Morne Rouge, France'
df.loc[df['Location Isolated'] == 'Vauclin, Martinique Island, French West Indes, France', 'Location Isolated'] = 'Vauclin, France'
df.loc[df['Location Isolated'] == 'Prêcheur, Martinique Island, French West Indes, France', 'Location Isolated'] = 'Prêcheur, France'
df.loc[df['Location Isolated'] == 'Saint Esprit, Martinique Island, French West Indes, France', 'Location Isolated'] = 'Saint Esprit, France'
df.loc[df['Location Isolated'] == 'Sainte Marie, Martinique Island, French West Indes, France', 'Location Isolated'] = 'Sainte Marie, France'
df.loc[df['Location Isolated'] == 'Thallande, Ethiopia', 'Location Isolated'] = 'Ethiopia'
df.loc[df['Location Isolated'] == 'Honshu, Kyoshu, Japan', 'Location Isolated'] = 'Kyoshu, Japan'
df.loc[df['Location Isolated'] == 'Petoseed/Hygrotech BW field trial site, South Africa', 'Location Isolated'] = 'South Africa'
df.loc[df['Location Isolated'] == 'Transvaal Lowveld, South Africa', 'Location Isolated'] = 'Mpumalanga, South Africa'
df.loc[df['Location Isolated'] == 'Souandala P2, Ivory Coast', 'Location Isolated'] = 'Sandala, Ivory Coast'
df.loc[df['Location Isolated'] == 'Alamo, Tecun Uman, Guatemala', 'Location Isolated'] = 'Ayutla, San Marcos, Guatemala'
df.loc[df['Location Isolated'] == 'Mojarras, Coatepeque, Guatemala', 'Location Isolated'] = 'Coatepeque, Guatemala'
df.loc[df['Location Isolated'] == 'Alamo,Tecun Uman, Guatemala', 'Location Isolated'] = 'Ayutla, San Marcos, Guatemala'
df.loc[df['Location Isolated'] == 'Coari (Amazon basin), Brazil', 'Location Isolated'] = 'Coari, Brazil'
df.loc[df['Location Isolated'] == 'Datta P2, Ivory Coast', 'Location Isolated'] = 'Ivory Coast'
df.loc[df['Location Isolated'] == 'Datta P1, Ivory Coast', 'Location Isolated'] = 'Ivory Coast'
df.loc[df['Location Isolated'] == 'Case Pilote, Martinique Island, French West Indes, France (France)', 'Location Isolated'] = 'Case Pilote, Martinique'
df.loc[df['Location Isolated'] == 'Case Pilote, Martinique Island, French West Indes, France', 'Location Isolated'] = 'Case Pilote, Martinique'
df.loc[df['Location Isolated'] == 'La chincana, San Ramon, Peru', 'Location Isolated'] = 'San Ramon, Peru'
df.loc[df['Location Isolated'] == 'Saint François, Guadeloupe Island, French West Indes,  (France)', 'Location Isolated'] = 'Saint François, France'
df.loc[df['Location Isolated'] == 'Saint François, Guadeloupe Island, French West Indes,', 'Location Isolated'] = 'Saint François, France'
df.loc[df['Location Isolated'] == 'CNPH-DF, Brazil', 'Location Isolated'] = 'Brazil'
df.loc[df['Location Isolated'] == 'Dschang (ZAE3), Cameroon', 'Location Isolated'] = 'Dschang, Cameroon'
df.loc[df['Location Isolated'] == 'Uruguay, Uruguay (xxcheck these)', 'Location Isolated'] = 'Uruguay'
df.loc[df['Location Isolated'] == 'Yaoundé (V) area V, Cameroon', 'Location Isolated'] = 'Yaoundé, Cameroon'
df.loc[df['Location Isolated'] == 'Amapa, Brazil, Brazil', 'Location Isolated'] = 'Amapa, Brazil'
df.loc[df['Location Isolated'] == 'Andimeshk, Khoozestan, Iran', 'Location Isolated'] = 'Andimeshk, Iran'
df.loc[df['Location Isolated'] == 'Fasarod, Fars, Iran', 'Location Isolated'] = 'Fars Province, Iran'
df.loc[df['Location Isolated'] == 'Dorche, Esfahan, Iran', 'Location Isolated'] = 'Isfahan Province, Iran'
df.loc[df['Location Isolated'] == 'Fereidan, Esfahan, Iran', 'Location Isolated'] = 'Fereydan, Iran'
df.loc[df['Location Isolated'] == 'Norabad, Fars, Iran', 'Location Isolated'] = 'Nurabad, Iran'
df.loc[df['Location Isolated'] == 'Maharlo, Fars, Iran', 'Location Isolated'] = 'Fars Province, Iran'
df.loc[df['Location Isolated'] == 'Guadeloupe Island, French West Indes,', 'Location Isolated'] = 'Guadeloupe Island'
df.loc[df['Location Isolated'] == 'Mol-Raw, Belgium', 'Location Isolated'] = 'Mol, Belgium'
df.loc[df['Location Isolated'] == 'Salamanca, Tormes River, Castilla-Leon, Spain', 'Location Isolated'] = 'Salamanca, Castilla-Leon, Spain'
df.loc[df['Location Isolated'] == 'Alfiaco, Cutervo, Cajamarca, Peru', 'Location Isolated'] = 'Cutervo, Cajamarca, Peru'
df.loc[df['Location Isolated'] == 'Huayao, Huancayo, Junín, Peru', 'Location Isolated'] = 'Huancayo, Junín, Peru'
df.loc[df['Location Isolated'] == 'Sihuas, Huaraz, Aneash, Peru', 'Location Isolated'] = 'Huaraz, Aneash, Peru'
df.loc[df['Location Isolated'] == 'Taskia 502, Niger', 'Location Isolated'] = 'Niger'
df.loc[df['Location Isolated'] == 'Toulloua, Niger', 'Location Isolated'] = 'Niger'
df.loc[df['Location Isolated'] == 'Bogol Hima, Niger', 'Location Isolated'] = 'Boboye, Dosso Region, Niger'
df.loc[df['Location Isolated'] == 'Ghajipur, Uttar Pradesh, India', 'Location Isolated'] = 'Ghazipur, Uttar Pradesh, India'
df.loc[df['Location Isolated'] == 'Bardwan, West Bengal, India', 'Location Isolated'] = 'Bardhaman, West Bengal, India'
df.loc[df['Location Isolated'] == 'Baham (ZAE3), Cameroon', 'Location Isolated'] = 'Baham, Cameroon'
df.loc[df['Location Isolated'] == 'Pietersberg, South Africa', 'Location Isolated'] = 'Polokwane, South Africa'
df.loc[df['Location Isolated'] == 'Jalapa, Guatamala, Guatemala', 'Location Isolated'] = 'Jalapa, Guatemala'
df.loc[df['Location Isolated'] == 'San Juan Alzatate, Jalapa, Guatemala', 'Location Isolated'] = 'San Carlos Alzatate, Jalapa, Guatemala'
df.loc[df['Location Isolated'] == 'San Juan Alzatate, Guatemala', 'Location Isolated'] = 'San Carlos Alzatate, Jalapa, Guatemala'
df.loc[df['Location Isolated'] == 'Miralmundo, Jalapa, Guatemala', 'Location Isolated'] = 'Miramundo, Jalapa, Guatemala'
df.loc[df['Location Isolated'] == 'Miralmundo Jalapa, Guatemala', 'Location Isolated'] = 'Miramundo, Jalapa, Guatemala'
df.loc[df['Location Isolated'] == 'La Chinana, San Ramón, Junín, Peru', 'Location Isolated'] = 'San Ramón, Junín, Peru'
df.loc[df['Location Isolated'] == 'Eastern plains region, Colombia', 'Location Isolated'] = 'Los Llanos, Colombia'
df.loc[df['Location Isolated'] == 'Southwestern Iran', 'Location Isolated'] = 'South, Iran'
df.loc[df['Location Isolated'] == 'La Chinana, Junín, Peru', 'Location Isolated'] = 'San Ramón, Junín, Peru'
df.loc[df['Location Isolated'] == 'Saint George County, Trinidad and Tobago', 'Location Isolated'] = 'Trinidad and Tobago'
df.loc[df['Location Isolated'] == 'Arima, St George County, Trinidad', 'Location Isolated'] = 'Trinidad and Tobago'
df.loc[df['Location Isolated'] == 'Mindana Island, Philippines', 'Location Isolated'] = 'Mindanao Island, Philippines'
df.loc[df['Location Isolated'] == 'Basse-Pointe, Martinique Island, French West Indes, France', 'Location Isolated'] = 'Basse-pointe, France'
df.loc[df['Location Isolated'] == 'Ste Anne, Martinique Island, French West Indes, France', 'Location Isolated'] = 'Sainte-Anne, France'
df.loc[df['Location Isolated'] == 'Morne rouge, Martinique Island, French West Indes, France', 'Location Isolated'] = 'Morne rouge, France'
df.loc[df['Location Isolated'] == 'François, Martinique Island, French West Indes, France', 'Location Isolated'] = 'François, France'
df.loc[df['Location Isolated'] == 'Saint-Joseph, Martinique Island, French West Indes, France', 'Location Isolated'] = 'Saint-Joseph, France'
df.loc[df['Location Isolated'] == 'Lézarde River, Gros-Morne, Martinique Island, French West Indes, France (France)', 'Location Isolated'] = 'Lézarde River, France'
df.loc[df['Location Isolated'] == 'Saint-Esprit, Martinique Island, French West Indes, France', 'Location Isolated'] = 'Saint-Esprit, France'
df.loc[df['Location Isolated'] == 'Ste Marie, Martinique Island, French West Indes, France', 'Location Isolated'] = 'Saint-Marie, France'
df.loc[df['Location Isolated'] == 'Saint Pierre, Martinique Island, French West Indes, France', 'Location Isolated'] = 'Saint-Pierre, France'
df.loc[df['Location Isolated'] == 'St Esprit, Martinique Island, French West Indes, France', 'Location Isolated'] = 'Saint-Esprit, France'
df.loc[df['Location Isolated'] == 'Basse Pointe, Martinique Island, French West Indes, France', 'Location Isolated'] = 'Basse-pointe, France'
df.loc[df['Location Isolated'] == 'Fond Saint Denis, Martinique Island, French West Indes, France', 'Location Isolated'] = 'Fond Saint Denis, France'
df.loc[df['Location Isolated'] == 'Marin, Martinique Island, French West Indes, France', 'Location Isolated'] = 'Marin, France'
df.loc[df['Location Isolated'] == 'Gasparillo, Trinidad', 'Location Isolated'] = 'Gasparillo, Trinidad and Tobago'
df.loc[df['Location Isolated'] == 'Shushtar, Khoozestan, Iran', 'Location Isolated'] = 'Shushtar, Iran'
df.loc[df['Location Isolated'] == 'Dezful, Khoozestan, Iran', 'Location Isolated'] = 'Dezful, Iran'
df.loc[df['Location Isolated'] == 'Le François, Martinique Island, French West Indes, France', 'Location Isolated'] = 'François, France'
df.loc[df['Location Isolated'] == 'Le Vauclin, Martinique Island, French West Indes, France', 'Location Isolated'] = 'Vauclin, France'
df.loc[df['Location Isolated'] == 'Rivière-Pilote, Martinique Island, French West Indes, France', 'Location Isolated'] = 'Rivière-Pilote, France'
df.loc[df['Location Isolated'] == 'Le Prêcheur, Martinique Island, French West Indes, France', 'Location Isolated'] = 'Prêcheur, France'
df.loc[df['Location Isolated'] == 'Ajoupa-Bouillon, Martinique Island, French West Indes, France', 'Location Isolated'] = 'Ajoupa-Bouillon, France'
df.loc[df['Location Isolated'] == 'Brasília, state - Federal District, Brazil', 'Location Isolated'] = 'Brasília, Brazil'
df.loc[df['Location Isolated'] == 'Contenda, Paraná state, Brazil', 'Location Isolated'] = 'Contenda, Brazil'
df.loc[df['Location Isolated'] == 'Canindé de São Franciscoé de San Francisco, Brazil', 'Location Isolated'] = 'Canindé de São Franciscoé, Brazil'
df.loc[df['Location Isolated'] == 'Cotinguiba Pindoba, Brazil', 'Location Isolated'] = 'São Francisco River, Brazil'
df.loc[df['Location Isolated'] == 'Tan Rouge 700m, Reunion Island, France', 'Location Isolated'] = 'Tan Rouge, France'
df.loc[df['Location Isolated'] == 'Atochi (PA Banki), Cameroon', 'Location Isolated'] = 'Atochi, Cameroon'
df.loc[df['Location Isolated'] == 'Sonké Togueta, Guinea', 'Location Isolated'] = 'Guinea'
df.loc[df['Location Isolated'] == 'Ballayara, Niger', 'Location Isolated'] = 'Niger'
df.loc[df['Location Isolated'] == 'Bafang (ZAE3), Cameroon', 'Location Isolated'] = 'Bafang, Cameroon'
df.loc[df['Location Isolated'] == 'Trois Bassins 950m, Reunion Island, France', 'Location Isolated'] = 'Les Trois-Bassins, France'
df.loc[df['Location Isolated'] == 'Ginchi, Ethiopia', 'Location Isolated'] = 'West Shewa, Oromia, Ethiopia'
df.loc[df['Location Isolated'] == 'Dedo, Ethiopia', 'Location Isolated'] = 'Oromia, Ethiopia'
df.loc[df['Location Isolated'] == 'Camocin S. Felix, Brazil', 'Location Isolated'] = 'Camocim de São Félix, Brazil'
df.loc[df['Location Isolated'] == 'Tikurwuha, Ethiopia', 'Location Isolated'] = 'Ethiopia'
df.loc[df['Location Isolated'] == 'Kadusa, Minamishimabara, Nagasaki, Japan', 'Location Isolated'] = 'Minamishimabara, Nagasaki, Japan'
df.loc[df['Location Isolated'] == 'Aino, Unzen, Nagasaski, Japan', 'Location Isolated'] = 'Aino, Nagasaski, Japan'
df.loc[df['Location Isolated'] == 'Kitaarima, Minamishimabara, Nagasaki, Japan', 'Location Isolated'] = 'Minamishimabara, Nagasaki, Japan'
df.loc[df['Location Isolated'] == 'Kigatsu, Hirado, Nagasaki, Japan', 'Location Isolated'] = 'Hirado, Nagasaki, Japan'
df.loc[df['Location Isolated'] == 'Dhaérah Istiméwa Yogyakarta, Indonesia', 'Location Isolated'] = 'Yogyakarta, Indonesia'
df.loc[df['Location Isolated'] == 'Gendu, Jatimulyo, Yogyakarta', 'Location Isolated'] = 'Yogyakarta, Indonesia'
df.loc[df['Location Isolated'] == 'Canada (imported from Central America)', 'Location Isolated'] = 'Central America'
df.loc[df['Location Isolated'] == 'United States (imported from Guatemala)', 'Location Isolated'] = 'Guatemala'
df.loc[df['Location Isolated'] == 'Florida (imported from Guatemala)', 'Location Isolated'] = 'Guatemala'
df.loc[df['Location Isolated'] == 'United States (imported from Kenya)', 'Location Isolated'] = 'Kenya'
df.loc[df['Location Isolated'] == 'Neuva Ecija, Philippines', 'Location Isolated'] = 'Philippines'
df.loc[df['Location Isolated'] == 'Juneliagaon, Nepal', 'Location Isolated'] = 'Nepal'
df.loc[df['Location Isolated'] == 'Raijung, Nepal', 'Location Isolated'] = 'Nepal'
df.loc[df['Location Isolated'] == 'Japan, Thailand', 'Location Isolated'] = 'Japan'
df.loc[df['Location Isolated'] == 'Zanzibar, Tanzinia', 'Location Isolated'] = 'Tanzinia'
df.loc[df['Location Isolated'] == 'Nord Queensland, Australia', 'Location Isolated'] = 'North Queensland, Australia'
df.loc[df['Location Isolated'] == 'Itacoitiara, Amazonas, Brazil', 'Location Isolated'] = 'Niterói, Rio de Janeiro, Brazil'
df.loc[df['Location Isolated'] == 'Guepus, Costa Rica', 'Location Isolated'] = 'Quepos, Costa Rica'
df.loc[df['Location Isolated'] == 'Guadeloupe Island, French West Indes,', 'Location Isolated'] = ''
df.loc[df['Location Isolated'] == 'Matouba, Guadeloupe Island, French West Indes,', 'Location Isolated'] = 'Matouba, France'
df.loc[df['Location Isolated'] == 'Petit Bourg, Guadeloupe Island, French West Indes,', 'Location Isolated'] = 'Petit Bourg, France'
df.loc[df['Location Isolated'] == 'Saint François, Guadeloupe Island, French West Indes,', 'Location Isolated'] = 'Saint François, France'
df.loc[df['Location Isolated'] == 'Ligne Paradis 150m, Reunion Island, France', 'Location Isolated'] = 'Ligne Paradis, France'
df.loc[df['Location Isolated'] == 'Notre Dame de la Paix 1500m, Reunion Island, France', 'Location Isolated'] = 'Notre Dame de la Paix, France'
df.loc[df['Location Isolated'] == 'Piton Mont Vert 500m, Reunion Island, France', 'Location Isolated'] = 'Piton Mont Vert, France'
df.loc[df['Location Isolated'] == 'Plaine des Cafres 1600m, Reunion Island, France', 'Location Isolated'] = 'Plaine des Cafres, France'
df.loc[df['Location Isolated'] == 'Carbet, Martinique Island, French West Indes, France', 'Location Isolated'] = 'Carbet, France'
df.loc[df['Location Isolated'] == 'Martinique Island, French West Indes, France (France)', 'Location Isolated'] = 'Martinique Island, France'
df.loc[df['Location Isolated'] == 'Morne l´Eoile, Martinique Island, French West Indes, France', 'Location Isolated'] = 'Morne l´Eoile, France'
df.loc[df['Location Isolated'] == 'Sainte Lucie, Martinique Island, French West Indes, France', 'Location Isolated'] = 'Sainte Lucie, France'
df.loc[df['Location Isolated'] == 'Lupuana, Peru', 'Location Isolated'] = 'Peru'
df.loc[df['Location Isolated'] == 'San Ramson, Peru', 'Location Isolated'] = 'Peru'
df.loc[df['Location Isolated'] == 'Coari, Amazonas, Brazil, Amazonas, Brazil', 'Location Isolated'] = 'Coari, Brazil'
df.loc[df['Location Isolated'] == 'Mana, French Guiana, France', 'Location Isolated'] = 'Mana, France'
df.loc[df['Location Isolated'] == 'Doi Poo Muan, Fang, Chiang Mai, Thailand', 'Location Isolated'] = 'Chiang Mai, Thailand'
df.loc[df['Location Isolated'] == 'Hort. Res. Station, Kaoko, Petchabun, Thailand', 'Location Isolated'] = 'Petchabun, Thailand'
df.loc[df['Location Isolated'] == 'Hueysithon, Fang, Chiang Mai, Thailand', 'Location Isolated'] = 'Chiang Mai, Thailand'
df.loc[df['Location Isolated'] == 'Kasetsart market, Bangkok, Thailand', 'Location Isolated'] = 'Bangkok, Thailand'
df.loc[df['Location Isolated'] == 'Kok, Mae Sot Tak, Thailand', 'Location Isolated'] = 'Mae Sot, Thailand'
df.loc[df['Location Isolated'] == 'Mude, Sindhupalcok, Nepal', 'Location Isolated'] = 'Sindhupalchok, Nepal'
df.loc[df['Location Isolated'] == 'Nala, Kavrepalchok, Nepal', 'Location Isolated'] = 'Kavrepalanchok, Nepal'
df.loc[df['Location Isolated'] == 'Nigale, Sindhupalcok, Nepal', 'Location Isolated'] = 'Sindhupalchok, Nepal'
df.loc[df['Location Isolated'] == 'Noungmaloa, Lumpun, Thailand', 'Location Isolated'] = 'Lamphun, Thailand'
df.loc[df['Location Isolated'] == 'Nucleus Seed Potato Center, Sindhupalcok, Nepal', 'Location Isolated'] = 'Sindhupalchok, Nepal'
df.loc[df['Location Isolated'] == 'Civuiti, Democratic Republic of Congo', 'Location Isolated'] = 'Democratic Republic of Congo'
df.loc[df['Location Isolated'] == 'Dendron, Kwa-Zulu Natal, South Africa', 'Location Isolated'] = 'KwaZulu-Natal, South Africa'
df.loc[df['Location Isolated'] == 'Hlabisha, Kwa-Zulu Natal, South Africa', 'Location Isolated'] = 'KwaZulu-Natal, South Africa'
df.loc[df['Location Isolated'] == 'Kissoko, Democratic Republic of Congo', 'Location Isolated'] = 'Democratic Republic of Congo'
df.loc[df['Location Isolated'] == 'Rooiwal, Kwa-Zulu Natal, South Africa', 'Location Isolated'] = 'KwaZulu-Natal, South Africa'
df.loc[df['Location Isolated'] == 'Mioli, Taiwan', 'Location Isolated'] = 'Miaoli, Taiwan'
df.loc[df['Location Isolated'] == 'Adam Tulu, Ethiopia', 'Location Isolated'] = 'Ethiopia'
df.loc[df['Location Isolated'] == 'Arata Chaffa, Ziway, Ethiopia', 'Location Isolated'] = 'Batu, Ethiopia'
df.loc[df['Location Isolated'] == 'Dedo, Jimma, Ethiopia', 'Location Isolated'] = 'Jimma, Ethiopia' 
df.loc[df['Location Isolated'] == 'Ginchi, Ethiopia', 'Location Isolated'] = 'Oromia, Ethiopia'
df.loc[df['Location Isolated'] == 'Goromt, Ambo, Ethiopia', 'Location Isolated'] = 'Ambo, Ethiopia'
df.loc[df['Location Isolated'] == 'Jato, Gedo, Ethiopia', 'Location Isolated'] = 'Gedo, Ethiopia'
df.loc[df['Location Isolated'] == 'Karafildicha, Shashemene, Ethiopia', 'Location Isolated'] = 'Shashemene Zuria, Ethiopia'
df.loc[df['Location Isolated'] == 'Kejo, Ethiopia', 'Location Isolated'] = 'Ethiopia'
df.loc[df['Location Isolated'] == 'Kombolcha, east Hararge, Ethiopia', 'Location Isolated'] = 'Kombolcha, Ethiopia'
df.loc[df['Location Isolated'] == 'Mutulu, Guder, Ethiopia', 'Location Isolated'] = 'Guder, Ethiopia'
df.loc[df['Location Isolated'] == 'Qarsa, east Hararge, Ethiopia', 'Location Isolated'] = 'East Hararghe, Ethiopia'
df.loc[df['Location Isolated'] == 'Tseday, Holeta, Ethiopia', 'Location Isolated'] = 'Holeta, Ethiopia'
df.loc[df['Location Isolated'] == 'Tsedey, Holeta, Ethiopia', 'Location Isolated'] = 'Holeta, Ethiopia'
df.loc[df['Location Isolated'] == 'Karo Highland, Indonesia', 'Location Isolated'] = 'North Sumatra, Indonesia'
df.loc[df['Location Isolated'] == 'Cajamba, Cajamarca, Peru', 'Location Isolated'] = 'Cajamarca, Peru'
df.loc[df['Location Isolated'] == 'San Miguel, San Ramón, Junín, Peru', 'Location Isolated'] = 'San Ramón, Junín, Peru'
df.loc[df['Location Isolated'] == 'Coochebehar, West Bengal, India', 'Location Isolated'] = 'West Bengal, India'
df.loc[df['Location Isolated'] == 'Urabá, Valle del Cauca y Quindío, Colombia', 'Location Isolated'] = 'Valle del Cauca, Colombia'
df.loc[df['Location Isolated'] == 'Valle del Cauca y Quindío, Colombia', 'Location Isolated'] = 'Valle del Cauca, Colombia'
df.loc[df['Location Isolated'] == 'Burma Valley, Zimbabwe', 'Location Isolated'] = 'Zimbabwe'
df.loc[df['Location Isolated'] == 'Eumsung, South Korea', 'Location Isolated'] = 'Eumseong-gun, South Korea'
df.loc[df['Location Isolated'] == 'Hongchon, South Korea', 'Location Isolated'] = 'Hongcheon-gun, South Korea'
df.loc[df['Location Isolated'] == 'Ultarakhand, India', 'Location Isolated'] = 'Uttarakhand, India'
df.loc[df['Location Isolated'] == 'Sergipe, Northeastern Brazil', 'Location Isolated'] = 'Sergipe, Brazil'
df.loc[df['Location Isolated'] == 'Martinique Island, French West Indes, France, French West Indes', 'Location Isolated'] = 'Martinique Island, France'
df.loc[df['Location Isolated'] == 'Foot hills of Haldwani, Nainital, India', 'Location Isolated'] = 'Haldwani, India'
df.loc[df['Location Isolated'] == 'Guiyang and Nanxiong, China', 'Location Isolated'] = 'Guiyang, China'
df.loc[df['Location Isolated'] == 'Eumsong, South Korea', 'Location Isolated'] = 'Eumseong-gun, South Korea'
df.loc[df['Location Isolated'] == 'Wyanad Meppadi, India', 'Location Isolated'] = 'Meppadi, India'
df.loc[df['Location Isolated'] == 'France (probably not continental)', 'Location Isolated'] = 'France'
df.loc[df['Location Isolated'] == 'Tsu City, Mie Prefecture, Japan', 'Location Isolated'] = 'Tsu, Japan'
df.loc[df['Location Isolated'] == 'Kerman-Laleh Zaar, Iran', 'Location Isolated'] = 'Lalehzar, Iran'
df.loc[df['Location Isolated'] == 'Georgia, United States (Continental)', 'Location Isolated'] = 'Georgia, United States'
df.loc[df['Location Isolated'] == 'Longxi, Fujiang, China', 'Location Isolated'] = 'Longxi, China'
df.loc[df['Location Isolated'] == "Tai'an, Shangdong, China", 'Location Isolated'] = "Tai'an, China"
df.loc[df['Location Isolated'] == 'Coconi, Mayotte Island, France', 'Location Isolated'] = 'Mayotte, France'
df.loc[df['Location Isolated'] == 'Dembeni, Mayotte Island, France', 'Location Isolated'] = 'Mayotte, France'
df.loc[df['Location Isolated'] == "M'romouhou, Mayotte Island, France", 'Location Isolated'] = 'Mayotte, France'
df.loc[df['Location Isolated'] == 'Miangani, Mayotte Island, France', 'Location Isolated'] = 'Mayotte, France'
df.loc[df['Location Isolated'] == 'Combani, Mayotte Island, France', 'Location Isolated'] = 'Mayotte, France'
df.loc[df['Location Isolated'] == 'Mitseni, Mayotte Island, France', 'Location Isolated'] = 'Mayotte, France'
df.loc[df['Location Isolated'] == 'Gharbia, KafarAlziyat, Egypt', 'Location Isolated'] = 'Kafr El-Zayat, Egypt'
df.loc[df['Location Isolated'] == 'Gharbia, Basyoun, Egypt', 'Location Isolated'] = 'Basioun, Egypt'
df.loc[df['Location Isolated'] == 'Assiut, Dairout, Egypt', 'Location Isolated'] = 'Dairut, Egypt'
df.loc[df['Location Isolated'] == 'Assiut, Manfalout, Egypt', 'Location Isolated'] = 'Manfalut, Egypt'
df.loc[df['Location Isolated'] == 'Sohag, Akhmem, Egypt', 'Location Isolated'] = 'Akhmim, Egypt'
df.loc[df['Location Isolated'] == 'Sohag, El-Osayrat, Egypt', 'Location Isolated'] = 'El Usayrat, Egypt'
df.loc[df['Location Isolated'] == 'Tchouloum, Togo', 'Location Isolated'] = 'Togo'
df.loc[df['Location Isolated'] == 'Ceco-Agro, Togo', 'Location Isolated'] = 'Togo'
df.loc[df['Location Isolated'] == 'Yaoundé (V) area II, Cameroon', 'Location Isolated'] = 'Yaoundé, Cameroon'
df.loc[df['Location Isolated'] == 'Yaoundé (V) area IV, Cameroon', 'Location Isolated'] = 'Yaoundé, Cameroon'
df.loc[df['Location Isolated'] == 'Yaoundé (V) area I, Cameroon', 'Location Isolated'] = 'Yaoundé, Cameroon'
df.loc[df['Location Isolated'] == 'Puerto Algeria, Peru', 'Location Isolated'] = 'Puerto Alegria, Peru'
df.loc[df['Location Isolated'] == 'Palamar, Costa Rica', 'Location Isolated'] = 'Palmar, Costa Rica'
df.loc[df['Location Isolated'] == 'Ananlanjirofo, Madagascar', 'Location Isolated'] = 'Analanjirofo, Madagascar'
df.loc[df['Location Isolated'] == 'Muratpur (West Bengal)', 'Location Isolated'] = 'Muradpur, India'
df.loc[df['Location Isolated'] == 'Aistala (West Bengal)', 'Location Isolated'] = 'Ranaghat, India'
df.loc[df['Location Isolated'] == 'Mayakol (West Bengal)', 'Location Isolated'] = 'West Bengal, India'
df.loc[df['Location Isolated'] == 'Mokrampur (West Bengal)', 'Location Isolated'] = 'West Bengal, India'
df.loc[df['Location Isolated'] == 'Sasidharpur (West Bengal)', 'Location Isolated'] = 'West Bengal, India'
df.loc[df['Location Isolated'] == 'Rezinagar (West Bengal)', 'Location Isolated'] = 'West Bengal, India'
df.loc[df['Location Isolated'] == 'Baburgram (West Bengal)', 'Location Isolated'] = 'West Bengal, India'
df.loc[df['Location Isolated'] == 'Badhia (West Bengal)', 'Location Isolated'] = 'West Bengal, India'
df.loc[df['Location Isolated'] == 'Chak Dahari (West Bengal)', 'Location Isolated'] = 'West Bengal, India'
df.loc[df['Location Isolated'] == 'Khaurdaha (West Bengal)', 'Location Isolated'] = 'West Bengal, India'
df.loc[df['Location Isolated'] == 'Jahangirpur (West Bengal)', 'Location Isolated'] = 'West Bengal, India'
df.loc[df['Location Isolated'] == 'Gagnabadh (West Bengal)', 'Location Isolated'] = 'West Bengal, India'
df.loc[df['Location Isolated'] == 'Palasdanga (West Bengal)', 'Location Isolated'] = 'West Bengal, India'
df.loc[df['Location Isolated'] == 'Allampur (West Bengal)', 'Location Isolated'] = 'West Bengal, India'
df.loc[df['Location Isolated'] == 'Huda (West Bengal)', 'Location Isolated'] = 'West Bengal, India'
df.loc[df['Location Isolated'] == 'RSS (West Bengal)', 'Location Isolated'] = 'West Bengal, India'
df.loc[df['Location Isolated'] == 'Magfala (West Bengal)', 'Location Isolated'] = 'West Bengal, India'
df.loc[df['Location Isolated'] == 'Kamrganga (West Bengal)', 'Location Isolated'] = 'West Bengal, India'
df.loc[df['Location Isolated'] == 'Fuzhou North Peak', 'Location Isolated'] = 'Fuzhou, China'
df.loc[df['Location Isolated'] == 'Putian Dongyang', 'Location Isolated'] = 'Putian, Fujian, China'
df.loc[df['Location Isolated'] == 'Fuzhou Yingkou', 'Location Isolated'] = 'Fuzhou, China'



In [9]:
df.loc[df['Location Isolated'] == 'Southeast China', 'Location Isolated'] = 'China'
df.loc[df['Location Isolated'] == 'Huai River Basin', 'Location Isolated'] = 'Xinyang, China'
df.loc[df['Location Isolated'] == 'Tobasco, Mexico', 'Location Isolated'] = 'Villahermosa, Mexico'
df.loc[df['Location Isolated'] == 'South Florida, United States', 'Location Isolated'] = 'Florida, United States'
df.loc[df['Location Isolated'] == 'South China', 'Location Isolated'] = 'China'
df.loc[df['Location Isolated'] == 'Central America', 'Location Isolated'] = 'Honduras'
df.loc[df['Location Isolated'] == 'North Brazil', 'Location Isolated'] = 'Brazil'
df.loc[df['Location Isolated'] == 'Southwest China', 'Location Isolated'] = 'China'
df.loc[df['Location Isolated'] == 'Cental America', 'Location Isolated'] = 'Honduras'
df.loc[df['Location Isolated'] == 'North Florida, United States', 'Location Isolated'] = 'Florida, United States'


In [20]:
import numpy as np
from collections import Counter
from plotly.subplots import make_subplots
from itertools import zip_longest
import plotly.graph_objects as go

# filter for locations
filtered_df = df[df['Location Isolated'].notnull()]

# group data and calculate strain count for each group
grouped_df = filtered_df.groupby(['latitude', 'longitude', 'Phylotype']).agg({
    'Strainname': lambda x: ', '.join(x.dropna().astype(str)),
    'Sequevar': lambda x: ', '.join(x.dropna().astype(str)),
    'Location Isolated': lambda x: Counter(x.dropna()).most_common(1)[0][0] if x.nunique() > 0 else np.nan
}).reset_index()


grouped_df['strain_count'] = grouped_df['Strainname'].apply(lambda x: len(x.split(', ')))

# combine strainname and sequevar into a single column 'Strain_Sequevar'
grouped_df['Strain_Sequevar'] = grouped_df.apply(lambda row: ', '.join([f'{strain} ({sequevar})' if sequevar else strain for strain, sequevar in zip_longest(row['Strainname'].split(', '), row['Sequevar'].split(', '))]), axis=1)

# phylotype colors
colors = {
    'I': 'rgb(0, 123, 255)',      # Blue
    'II': 'rgb(255, 150, 0)',     # Yellow
    'III': 'rgb(50, 205, 50)',    # Green
    'IV': 'rgb(227, 11, 193)'    # Purple
}

# creating plotly map w bubble size scaling
fig = make_subplots(specs=[[{'type': 'scattergeo'}]])

# normalized strain count for sizing
grouped_df['normalized_strain_count'] = (grouped_df['strain_count'] - grouped_df['strain_count'].min()) / (grouped_df['strain_count'].max() - grouped_df['strain_count'].min())

# offset factor for location coordinates
offset_factor = 0.1

# group data by 'latitude', 'longitude' and 'Phylotype'
grouped_location_phylotype = grouped_df.groupby(['latitude', 'longitude', 'Phylotype'])

# loop through groups
for (lat, lon, phylotype), group in grouped_location_phylotype:
    # skip group if it doesn't have valid Strainname or Sequevar
    if group['Strainname'].isna().all() and group['Sequevar'].isna().all():
        continue

    # create list of strains with sequevars 
    strains_with_sequevars_list = group['Strain_Sequevar'].str.split(', ', expand=True).stack().dropna().astype(str).tolist()

    # max characters for strains section of hover label
    max_label_chars = 500 

    # calculate the total number of characters
    total_chars = 0
    strains_displayed = []
    for strain in strains_with_sequevars_list:
        # calculate number of characters for strain (including line breaks)
        strain_chars = len(strain) + 4  
        if total_chars + strain_chars <= max_label_chars:
            strains_displayed.append(strain)
            total_chars += strain_chars
        else:
            break

    # combine displayed strains with line breaks
    strains_with_sequevars = '<br>'.join(strains_displayed)

    location_name = group['Location Isolated'].values[0]

    # skip if strains_with_sequevars is empty
    if not strains_with_sequevars.strip():
        continue

    # bubble size
    bubble_size = 5 + 15 * group['normalized_strain_count'].values[0]

    # transperent marker
    transparency = 0.5  

    # create marker color with transparency
    marker_color = colors.get(phylotype, 'black')
    marker_color_with_transparency = f'rgba({marker_color}, {transparency})'
    
     # offset factor applied to location coordinates
    phylotype_offset = {'I': (-offset_factor, -offset_factor),
                        'II': (offset_factor, -offset_factor),
                        'III': (-offset_factor, offset_factor),
                        'IV': (offset_factor, offset_factor)}
    lat_offset, lon_offset = phylotype_offset.get(phylotype, (0, 0))
    lat += lat_offset
    lon += lon_offset

    fig.add_trace(go.Scattergeo(
        lon=[lon],
        lat=[lat],
        text=f'Location: {location_name}<br>Phylotype: {phylotype}<br>Strain (Sequevar):<br>{strains_with_sequevars}',
        hoverinfo='text',
        mode='markers',
        marker=dict(color=marker_color, size=bubble_size, opacity=transparency),
        showlegend=False  # Add this line to hide the trace from the legend
    ))

for phylotype, color in colors.items():
    fig.add_trace(go.Scattergeo(
        lon=[None],  # No data for these traces
        lat=[None],
        mode='markers',
        marker=dict(color=color, size=10),
        name=f'Phylotype {phylotype}',
        legendgroup=f'Phylotype {phylotype}',
        showlegend=True
    ))

fig.update_layout(
    title_text='Strain Locations by Phylotype',
    showlegend=True,
    legend_title="Phylotypes",
    geo=dict(
        scope='world',
        projection_type='natural earth',
        showland=True,
        landcolor='rgb(243, 243, 243)',
        countrycolor='rgb(204, 204, 204)'
    )
)

fig.write_html('output_map_plotly.html')

In [16]:
import numpy as np
from collections import Counter
from plotly.subplots import make_subplots
from itertools import zip_longest
import plotly.graph_objects as go


filtered_df = df[df['Location Isolated'].notnull()]

grouped_df = filtered_df.groupby(['latitude', 'longitude', 'Phylotype']).agg({
    'Strainname': lambda x: ', '.join(x.dropna().astype(str)),
    'Sequevar': lambda x: ', '.join(x.dropna().astype(str)),
    'Location Isolated': lambda x: Counter(x.dropna()).most_common(1)[0][0] if x.nunique() > 0 else np.nan
}).reset_index()

# filter df to only include rows where Phylotype is 'I'
grouped_df = grouped_df[grouped_df['Phylotype'] == 'IV']

grouped_df['strain_count'] = grouped_df['Strainname'].apply(lambda x: len(x.split(', ')))

grouped_df['Strain_Sequevar'] = grouped_df.apply(lambda row: ', '.join([f'{strain} ({sequevar})' if sequevar else strain for strain, sequevar in zip_longest(row['Strainname'].split(', '), row['Sequevar'].split(', '))]), axis=1)

# phylotype colors
colors = {
    'IV': 'rgb(227, 11, 193)',     
}

fig = make_subplots(specs=[[{'type': 'scattergeo'}]])

grouped_df['normalized_strain_count'] = (grouped_df['strain_count'] - grouped_df['strain_count'].min()) / (grouped_df['strain_count'].max() - grouped_df['strain_count'].min())

offset_factor = 0.1

grouped_location_phylotype = grouped_df.groupby(['latitude', 'longitude', 'Phylotype'])

# loop through groups
for (lat, lon, phylotype), group in grouped_location_phylotype:
    # skip the group if it doesn't have valid Strainname or Sequevar
    if group['Strainname'].isna().all() and group['Sequevar'].isna().all():
        continue

    # create list of strains with sequevars
    strains_with_sequevars_list = group['Strain_Sequevar'].str.split(', ', expand=True).stack().dropna().astype(str).tolist()

    # max characters for strains section of the hover label
    max_label_chars = 500  # Adjust this value as needed

    # calculate the total number of characters
    total_chars = 0
    strains_displayed = []
    for strain in strains_with_sequevars_list:
        # calculate number of characters for strain (including line breaks)
        strain_chars = len(strain) + 4 
        if total_chars + strain_chars <= max_label_chars:
            strains_displayed.append(strain)
            total_chars += strain_chars
        else:
            break

    # combine displayed strains with line breaks
    strains_with_sequevars = '<br>'.join(strains_displayed)

    location_name = group['Location Isolated'].values[0]

    # skip if strains_with_sequevars is empty
    if not strains_with_sequevars.strip():
        continue

    # bubble size 
    bubble_size = 5 + 15 * grouped_df.loc[(grouped_df['latitude'] == lat) & (grouped_df['longitude'] == lon) & (grouped_df['Phylotype'] == phylotype), 'normalized_strain_count'].values[0]

    # transparent marker color
    transparency = 0.5 

    # create marker color with transparency
    marker_color = colors.get(phylotype, 'black')
    marker_color_with_transparency = f'rgba({marker_color}, {transparency})'
    
     # offset factor applied to location coordinates
    phylotype_offset = {'I': (-offset_factor, -offset_factor),
                        'II': (offset_factor, -offset_factor),
                        'III': (-offset_factor, offset_factor),
                        'IV': (offset_factor, offset_factor)}
    lat_offset, lon_offset = phylotype_offset.get(phylotype, (0, 0))
    lat += lat_offset
    lon += lon_offset

    fig.add_trace(go.Scattergeo(
        lon=[lon],
        lat=[lat],
        text=f'Location: {location_name}<br>Phylotype: {phylotype}<br>Strain (Sequevar):<br>{strains_with_sequevars}',
        hoverinfo='text',
        mode='markers',
        marker=dict(color=marker_color, size=bubble_size, opacity=transparency)
    ))


fig.update_layout(
    title_text='Strain Locations by Phylotype IV',
    showlegend=False,
    geo=dict(
        scope='world',
        projection_type='natural earth',
        showland=True,
        landcolor='rgb(243, 243, 243)',
        countrycolor='rgb(204, 204, 204)'
    )
)


fig.write_html('output_map_plotly_{phylotype IV}.html')
